---
title: "Funciones. Multiple dispatch"
subtitle: "Ejercicios"
---

# Ejercicios canónicos

## Sucesión de Fibonacci mediante recursión

Calcule el elemento $n$-ésimo de la sucesión de Fibonacci usando recursión

## Suma cuadrados

a) Escriba una función llamada `suma_cuadrados` que acepte el número natural $n$ y devuelva la suma $1^2 + 2^2 + \cdots + (n-1)^2 + n^2$

In [ ]:
@test suma_cuadrados(1) == 1
@test suma_cuadrados(2) == 9
@test suma_cuadrados(100) == 338350

b) Escribe una función llamada `fizz_buzz` que acepte un número y
- Si el número es divisible por 3 pero no 5, escriba "Fizz" (sin comillas)
- Si el número es divisible por 5 pero no 3, escriba "Buzz" (sin comillas)
- Si el número es divisible por 3 y 5, escriba "FizzBuzz" (sin comillas)
- En otro caso, escriba el propio número


In [ ]:
@test fizz_buzz(1) == 1
@test fizz_buzz(3) == "Fizz"
@test fizz_buzz(5) == "Buzz"
@test fizz_buzz(15) == "FizzBuzz"
@test fizz_buzz(16) == 16

# Creando un tipo de polinomios

## Estructuras {auto-animate=true}

Creamos una estructura para polinomios de coeficientes reales.

Basta con almacenar los coeficientes

In [ ]:
struct Poly
    coeff::Vector{Float64} # [a₀, a₁,... , aₙ]
end

## Multiple dispatch. Funciones sobre polinomios

Vamos a extender la funciones `iszero` y `+`. Para ello, creamos primero la función `degree`

In [ ]:
function degree(P::Poly)
    k = length(P.coeff)
    while k ≥ 1 && iszero(P.coeff[k])
        k = k - 1
    end
    return k - 1
end

Esto funciona para algunos polinomios, pero no para otros. Para esto 

Vamos a hacer una función `iszero` para polinomios

In [ ]:
import Base: iszero, isequal

function iszero(P::Poly)
    return (degree(P) < 0)
end

## Multiple dispatch. Operaciones

In [ ]:
#| code-line-numbers: "1-2|2-7|8-12|"

import Base: +
function +(a::Poly, b::Poly)
    if degree(a) < 0
        return b
    elseif degree(b) < 0
        return a
    else
        da = degree(a)
        db = degree(b)
        return Poly(
            [a.coeff[1:da+1]; zeros(max(db - da, 0))]
            +
            [b.coeff[1:db+1]; zeros(max(da - db, 0))]
        )
    end
end

. . .

In [ ]:
P = Poly([0,1,2])
print("P+P = ", P+P)

## Haciéndolo bonito

In [ ]:
function Base.show(io::IO,P::Poly) 
    degreeP = degree(P)
    if degreeP < 0
        text = "0"
    else
        text = "$(P.coeff[1])"
        for n = 2:degreeP + 1
            text = text * " + $(P.coeff[n])" * "x^$(n-1)"
        end
    end
    print(text)
    return nothing
end;

In [ ]:
P = Poly([0,1,2])
print(P)

## Otras operaciones

In [ ]:
P = Poly([0,1,0])
print("degree(P) = ", degree(P))

In [ ]:
#| code-line-numbers: "1-2,6-10"

import Base: *
function *(a::Poly,b::Poly)
    da = degree(a); db = degree(b);
    prodcoeffs = zeros(da+db+1);
    for k=0:(da+db)
        rangei = max(0,k-db):min(k,da)
        prodcoeffs[k+1] = sum(  a.coeff[1   .+ rangei] .*
                                b.coeff[k+1 .- rangei]);
    end
    return Poly(prodcoeffs)
end

. . .

In [ ]:
import Base: -, ==
function -(a::Poly, b::Poly)
    return a + Poly([-1]) * b
end

function ==(P::Poly, Q::Poly)
    return iszero(P - Q)
end

Ahora ya podemos usar 

In [ ]:
using Test

---

In [ ]:
P = Poly([0, 1, 2])
@test P * P == Poly([0.0, 0.0, 1.0, 4.0, 4.0])
@test P - P == Poly([0.0])

# El algoritmo de Euclides

<!-- Some introduction before -->


## Para números enteros

In [ ]:
function gcd(a, b)
    if iszero(b)
        return a
    else
        return gcd(b, a % b)
    end
end

@test gcd(30, 15) == 15

## Extensión a polinomios

Podemos extender `gcd` a cualquier lugar con `iszero` y `%`. 

. . .

Queremos ***extender*** la funciones anteriores anterior, de modo que `gcd` funcione ***sin cambios***

. . .

Por esto julia es uno de los lenguajes con más reciclaje de código. 

Creamos una estructura para polinomios de coeficientes reales.

Basta con almacenar los coeficientes.

## Añadiendo `%` a `Poly`

In [ ]:
import Base: %
function %(a::Poly, b::Poly)
    if degree(a) < 0 # a = 0
        error("¡No dividas por 0!")
    end

    r = a             # En cada paso a = b * q + r

    while degree(r) ≥ degree(b)
        s = Poly([zeros(degree(r) - degree(b)); lead(r) / lead(b)])
        r = r - s * b
    end

    return r
end

Se puede usar la siguiente función auxiliar

In [ ]:
function lead(P::Poly)
    if degree(P) < 0
        return 0.0
    else
        return P.coeff[degree(P)+1]
    end
end

. . .

In [ ]:
R = Poly([2])
Q = Poly([-3, 1])
P = Q * Q * Q + R

@test P % Q == R

## Grand finale

Lanzamos la función gcd ¡que nunca oyó hablar de polinomios! Y obtenemos 

. . .

In [ ]:
Q = Poly([-3, 1])
P = Q * Q * Q

@test gcd(P, Q) == Q